# BERT Viz

Took 32 minutes to run on full dataset. Use BII-GPU to speed up the process.

https://colab.research.google.com/drive/1r_eoi8CMea_a3YjWC1M4EmTqKMGVMbzQ?usp=sharing#scrollTo=afOPAzVoczRW

https://colab.research.google.com/drive/1DjtrD_MMW_Ezto0Q4zUvjT0IxKZg-rIt?usp=sharing#scrollTo=rYa_KDUTSprJ

In [2]:
#pgadmin
import os
import psycopg2 as pg


# For BERT
from transformers import DistilBertTokenizerFast, DistilBertModel

# For data manipulation and analysis
import pandas as pd
pd.options.display.max_colwidth = 200
import numpy as np
from sklearn.decomposition import PCA

# For interactive data visualization
import altair as alt


import re

In [3]:
#TODO: work on the cleaning function
def clean_text(texts):
    texts = re.sub(r'\[[0-9]*\]', ' ', texts) #removes numbers
   # texts = re.sub(r'\s+',' ',texts) #remove long blank space
    texts = texts.lower() #lowercase text
   # texts = re.sub(r'\d', ' ', texts) 
   # texts = re.sub(r'\s+', ' ', texts)
    return(texts)

# Data

In [4]:
repo_data = pd.read_csv("/home/zz3hs/git/dspg21oss/data/dspg21oss/github_repos_157k.csv")

In [35]:
#repo_data =repo_data_copy 

In [36]:
len(repo_data)

157538

In [37]:
#repo_data= repo_data.iloc[0:100000]
repo_data

,slug,description,readme,language,topics,commits,forks,stars,watchers
0,vuejs/vue,"🖖 Vue.js is a progressive, incrementally-adoptable JavaScript framework for building UI on the web.","<p align=""center""><a href=""https://vuejs.org"" rel=""noopener noreferrer"" target=""_blank""><img alt=""Vue logo"" src=""https://vuejs.org/images/logo.png"" width=""100""/></a></p>\n<p align=""center"">\n<a hr...",JavaScript,"['vue', 'javascript', 'frontend', 'framework']",3070,29611,185611,6250
1,facebook/react,"A declarative, efficient, and flexible JavaScript library for building user interfaces.",# [React](https://reactjs.org/) · [![GitHub license](https://img.shields.io/badge/license-MIT-blue.svg)](https://github.com/facebook/react/blob/main/LICENSE) [![npm version](https://img.shields.io...,JavaScript,"['javascript', 'react', 'frontend', 'declarative', 'ui', 'library']",12695,34352,171327,6718
2,tensorflow/tensorflow,An Open Source Machine Learning Framework for Everyone,"<div align=""center"">\n<img src=""https://www.tensorflow.org/images/tf_logo_social.png""/>\n</div>\n\n[![Python](https://img.shields.io/pypi/pyversions/tensorflow.svg?style=plastic)](https://badge.fu...",C++,"['tensorflow', 'machine-learning', 'python', 'deep-learning', 'deep-neural-networks', 'neural-network', 'ml', 'distributed']",75671,84937,156754,8092
3,twbs/bootstrap,"The most popular HTML, CSS, and JavaScript framework for developing responsive, mobile first projects on the web.","<p align=""center"">\n<a href=""https://getbootstrap.com/"">\n<img alt=""Bootstrap logo"" height=""165"" src=""https://getbootstrap.com/docs/5.0/assets/brand/bootstrap-logo-shadow.png"" width=""200""/>\n</a>\...",JavaScript,"['css', 'bootstrap', 'javascript', 'html', 'scss', 'css-framework', 'sass']",19228,73981,151778,7079
4,ohmyzsh/ohmyzsh,"🙃 A delightful community-driven (with 1700+ contributors) framework for managing your zsh configuration. Includes nearly 300 optional plugins (rails, git, OSX, hub, capistrano, brew, ant, php, py...","<p align=""center""><img alt=""Oh My Zsh"" src=""https://s3.amazonaws.com/ohmyzsh/oh-my-zsh-logo.png""/></p>\n\nOh My Zsh is an open source, community-driven framework for managing your [zsh](https://ww...",Shell,"['shell', 'zsh-configuration', 'theme', 'terminal', 'productivity', 'hacktoberfest']",5447,22232,129314,2678
...,...,...,...,...,...,...,...,...,...
157533,VeryLittleGravitas/CDTADPQ,Very Little Gravitas implementation of Prototype B for RFI # CDT-ADPQ-0117,"# CA Alerts, made with Very Little Gravitas for CDT RFI # CDT-ADPQ-0117\n\n[CA Alerts](https://alerts-ca.herokuapp.com) ([https://alerts-ca.herokuapp.com](https://alerts-ca.herokuapp.com)), submit...",CSS,"['prototype', 'messaging', 'emergency', 'govtech']",414,1,0,2
157534,dajinchu/kde-connect-android,For Google Code-In,NaN,Java,[],414,0,0,1
157535,LibrinnoTeam/LibraryHelpBot,Library Management System. ITP2 project,# Library Help Bot\r\n\r\n## Purpose of the application\r\n\r\nThis is the implementation of Library Management System (LMS). LMSs are used in libraries to track the different items in the library...,Python,"['telegram-bot', 'mariadb', 'python', 'flask']",415,2,0,2
157536,Twissi/Animator,Animator for hacklace. See hacklace.org for further information.,Animator\n========\n\nAnimator for hacklace. See hacklace.org for further information.\n\nInstallation steps\n==================\n\n1. Install and setup [maven](http://maven.apache.org/).\n2. Clon...,Java,[],415,0,0,0


In [7]:
# randomly select a subset
#repo_data = repo_data.sample(10000)

In [38]:
# create a list of description text
python_description_text = repo_data["description"].tolist()

#text cleaning
python_description_text_clean = []
for text in python_description_text:
    cleaned_text_i= clean_text(text)
    python_description_text_clean.append(cleaned_text_i)


In [39]:
len(python_description_text_clean)

157538

In [40]:
print(python_description_text_clean[0:10])

['🖖 vue.js is a progressive, incrementally-adoptable javascript framework for building ui on the web.', 'a declarative, efficient, and flexible javascript library for building user interfaces.', 'an open source machine learning framework for everyone', 'the most popular html, css, and javascript framework for developing responsive, mobile first projects on the web.', '🙃  a delightful community-driven (with 1700+ contributors) framework for managing your zsh configuration. includes nearly 300 optional plugins (rails, git, osx, hub, capistrano, brew, ant, php, python, etc), over 140 themes to spice up your morning, and an auto-update tool so that makes it easy to keep up with the latest updates from the community.', 'flutter makes it easy and fast to build beautiful apps for mobile and beyond.', '📝 algorithms and data structures implemented in javascript with explanations and links to further readings', 'javascript style guide', 'all algorithms implemented in python', '「java学习+面试指南」一份涵盖大

# Encode/tokenize text data for BERT

In [41]:
from transformers import DistilBertTokenizerFast

In [42]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

The `tokenizer()` will break word tokens into word pieces, truncate to 512 tokens, and add padding and special BERT tokens.

In [43]:
tokenized_description = tokenizer(python_description_text_clean, truncation=True, padding=True, return_tensors="pt")

In [44]:
' '.join(tokenized_description[0].tokens)

'[CLS] [UNK] vu ##e . j ##s is a progressive , inc ##rem ##ental ##ly - adopt ##able java ##script framework for building ui on the web . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD

## Load pre-trained BERT model

In [45]:
from transformers import DistilBertModel

In [46]:
#model = DistilBertModel.from_pretrained('distilbert-base-uncased').to("cuda")
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [48]:
import time
t1 = time.time()
print(time.asctime( time.localtime(time.time())))

print("Start time:",t1)
# List of vocabulary word IDs for all the words in each document (aka each repo description)
doc_word_ids = []
# List of word vectors for all the words in each document (aka each repo description)
doc_word_vectors = []

# Below we will slice our repo description to ignore the first (0th) and last (-1) special BERT tokens
start_of_words = 1
end_of_words = -1

# Below we will index the 0th or first document, which will be the only document, since we're analzying one repo description at a time
first_document = 0

for i, description in enumerate(python_description_text):
  
    # Here we tokenize each poem with the DistilBERT Tokenizer
    inputs = tokenizer(description, return_tensors="pt", truncation=True, padding=True)

    # Here we extract the vocabulary word ids for all the words in the poem (the first or 0th document, since we only have one document)
    # We ignore the first and last special BERT tokens
    # We also convert from a Pytorch tensor to a numpy array
    doc_word_ids.append(inputs.input_ids[first_document].numpy()[start_of_words:end_of_words])

    # Here we send the tokenized poems to the GPU
    # The model is already on the GPU, but this poem isn't, so we send it to the GPU
    inputs.to("cuda")
    
    # Here we run the tokenized poem through the DistilBERT model
    outputs = model(**inputs)

    # We take every element from the first or 0th document, from the 2nd to the 2nd to last position
    # Grabbing the last layer is one way of getting token vectors. There are different ways to get vectors with different pros and cons
    doc_word_vectors.append(outputs.last_hidden_state[first_document,start_of_words:end_of_words,:].detach().cpu().numpy())
t2 = time.time()
print("End time:",t2)
print(time.asctime( time.localtime(time.time()) ))
print("It took",t2-t1, "to run.")

Tue Jul 20 13:41:55 2021
Start time: 1626802915.251631
End time: 1626804844.0662737
Tue Jul 20 14:14:04 2021
It took 1928.8146426677704 to run.


In [49]:
#Confirm that we have the same number of documents for both the tokens and the vectors:
len(doc_word_ids), len(doc_word_vectors)

(157538, 157538)

In [50]:
doc_word_ids[0], doc_word_vectors[0]

(array([  100, 24728,  2063,  1012,  1046,  2015,  2003,  1037,  6555,
         1010,  4297, 28578, 21050,  2135,  1011, 11092,  3085,  9262,
        22483,  7705,  2005,  2311, 21318,  2006,  1996,  4773,  1012]),
 array([[-0.3178189 ,  0.22388487,  0.16467538, ...,  0.20646247,
          0.00553884,  0.21317537],
        [ 0.3150818 , -0.08535431,  0.5484929 , ...,  0.1767203 ,
         -0.00273554,  0.722559  ],
        [-0.18981865, -0.08054999,  0.04024157, ...,  0.04988308,
          0.10638252,  0.15953456],
        ...,
        [-0.54138   ,  0.11882845, -0.01523634, ..., -0.1729398 ,
         -0.08477003, -0.3005202 ],
        [-0.3491223 ,  0.01349686,  0.24578585, ..., -0.17824402,
          0.27983984, -0.5200093 ],
        [ 0.59265244,  0.04196135, -0.49857986, ...,  0.25213638,
         -0.7763895 , -0.3546484 ]], dtype=float32))

### Concatenate all word IDs/vectors for all doc

In [51]:
all_word_ids = np.concatenate(doc_word_ids)
all_word_vectors = np.concatenate(doc_word_vectors, axis=0)

In [52]:
# Calculating the length of each vector (Pythagorean theorem)
row_norms = np.sqrt(np.sum(all_word_vectors ** 2, axis=1))
# Dividing every vector by its length
all_word_vectors /= row_norms[:,np.newaxis]

## Find all word positions in a collection

In [53]:
def get_word_positions(words):
  
    """This function accepts a list of words, rather than a single word"""

    # Get word/vocabulary ID from BERT for each word
    word_ids = [tokenizer.vocab[word] for word in words]

    # Find all the positions where the words occur in the collection
    word_positions = np.where(np.isin(all_word_ids, word_ids))[0]

    return word_positions

## Find word from word position

In [54]:
# Here we create an array so that we can go backwards from numeric token IDs to words
word_lookup = np.empty(tokenizer.vocab_size, dtype="O")

for word, index in tokenizer.vocab.items():
    word_lookup[index] = word

In [55]:
def get_context(word_id, window_size=10):
  
    """Simply get the tokens that occur before and after word position"""

    start_pos = max(0, word_id - window_size) # The token where we will start the context view
    end_pos = min(word_id + window_size + 1, len(all_word_ids)) # The token where we will end the context view

    # Make a list called tokens and use word_lookup to get the words for given token IDs from starting position up to the keyword
    tokens = [word_lookup[word] for word in all_word_ids[start_pos:end_pos] ]

    context_words = " ".join(tokens)

    return context_words

In [56]:
def get_context_clean(word_id, window_size=10):
  
    """Get the tokens that occur before and after word position AND make them more readable"""

    keyword = word_lookup[all_word_ids[word_id]]
    start_pos = max(0, word_id - window_size) # The token where we will start the context view
    end_pos = min(word_id + window_size + 1, len(all_word_ids)) # The token where we will end the context view

    # Make a list called tokens and use word_lookup to get the words for given token IDs from starting position up to the keyword
    tokens = [word_lookup[word] for word in all_word_ids[start_pos:end_pos] ]

    # Make wordpieces slightly more readable
    # This is probably not the most efficient way to clean and correct for weird spacing
    context_words = " ".join(tokens)
    context_words = re.sub(r'\s+([##])', r'\1', context_words)
    context_words = re.sub(r'##', r'', context_words)
    context_words = re.sub('\s+\'s', '\'s', context_words)
    context_words = re.sub('\s+\'d', '\'d', context_words)
    context_words = re.sub('\s\'er', '\'er', context_words)
    context_words = re.sub(r'\s+([-,:?.!;])', r'\1', context_words)
    context_words = re.sub(r'([-\'"])\s+', r'\1', context_words)
    context_words = re.sub('\s+\'s', '\'s', context_words)
    context_words = re.sub('\s+\'d', '\'d', context_words)

    # Bold the keyword by putting asterisks around it
    if keyword in context_words:
        context_words = re.sub(f"\\b{keyword}\\b", f"**{keyword}**", context_words)
        context_words = re.sub(f"\\b({keyword}[esdtrlying]+)\\b", fr"**\1**", context_words)

    return context_words

## Viz

In [57]:
from IPython.display import Markdown, display

def print_md(string):
    display(Markdown(string))

## PCA

In [58]:
from sklearn.decomposition import PCA


### Merge with original data

## Plot embeddings

In [59]:
import altair as alt

# Plot word embeddings from keywords (all at once)

In [60]:
# Tokenize all the repo descriptions
tokenized_repos = tokenizer(python_description_text_clean, truncation=True, padding=True, return_tensors="pt")

# Get a list of all the tokens for each repo descriptions
all_tokenized_repos = []
for i in range(len(tokenized_repos['input_ids'])):
    all_tokenized_repos.append(' '.join(tokenized_repos[i].tokens))

# Add them to the original DataFrame
repo_data['tokens'] = all_tokenized_repos

In [61]:
def find_original_repo(rows):

    """This function checks to see whether the context tokens show up in the original repo description,
    and if so, returns metadata about the the slug, description, topic, stars of that repo"""

    text = rows['tokens'].replace('**', '')
    text = text[55:70]

    if repo_data['tokens'].str.contains(text, regex=False).any() == True :
        row = repo_data[repo_data['tokens'].str.contains(text, regex=False)].values[0]
        slug, description, topic, stars = row[0], row[1], row[7], row[6]
        return slug, description, topic, stars
    else:
        return None, None, None, None

In [64]:
t1 = time.time()
print("Start time:", t1)
#List of keywords that you want to compare
keywords = ['database', "machine","learning", "artificial", "intelligence","python", "r","ai","ml","visual"]
# "visualization", "blockchain", "cryptocurrency"
# How to color the points in the plot. The other option is "period" for time period
color_by = 'word'

# Get all word positions
word_positions = get_word_positions(keywords)

# Get all contexts around the words
keyword_contexts = []
keyword_contexts_tokens = []
words = []

for position in word_positions:
    words.append(word_lookup[all_word_ids[position]])
    keyword_contexts.append(get_context_clean(position))
    keyword_contexts_tokens.append(get_context(position))

# Reduce word vectors with PCA
pca = PCA(n_components=2)
pca.fit(all_word_vectors[word_positions,:].T)

# Make a DataFrame with PCA results
df = pd.DataFrame({"x": pca.components_[0,:], "y": pca.components_[1,:],
                   "context": keyword_contexts, "tokens": keyword_contexts_tokens, "word": words})
df[['slug', 'description', 'topic', 'stars']] = df.apply(find_original_repo, axis='columns', result_type='expand')

t2 = time.time()
print("End time:", t2)

In [65]:
df

,x,y,context,tokens,word,slug,description,topic,stars
0,-0.009454,-0.008559,"script library for building user interfaces. an open source **machine** learning framework for everyone the most popular html, cs","##script library for building user interfaces . an open source machine learning framework for everyone the most popular html , cs",machine,tensorflow/tensorflow,An Open Source Machine Learning Framework for Everyone,156754.0,84937.0
1,-0.009236,-0.011399,"library for building user interfaces. an open source machine **learning** framework for everyone the most popular html, css","library for building user interfaces . an open source machine learning framework for everyone the most popular html , cs ##s",learning,tensorflow/tensorflow,An Open Source Machine Learning Framework for Everyone,156754.0,84937.0
2,-0.009410,0.009145,"capistrano, brew, ant, php, **python**, etc ), over 140 themes to spice up","cap ##ist ##rano , brew , ant , php , python , etc ) , over 140 themes to spice up",python,ohmyzsh/ohmyzsh,"🙃 A delightful community-driven (with 1700+ contributors) framework for managing your zsh configuration. Includes nearly 300 optional plugins (rails, git, OSX, hub, capistrano, brew, ant, php, py...",129314.0,22232.0
3,-0.009638,0.011961,further readings javascript style guide all algorithms implemented in **python** 「 java 学 [UNK] + 面 [UNK] [UNK] 南 」,further readings java ##script style guide all algorithms implemented in python 「 java 学 [UNK] + 面 [UNK] [UNK] 南 」,python,TheAlgorithms/Python,All Algorithms implemented in Python,111656.0,30642.0
4,-0.010445,0.010868,html presentation framework the swift programming language spring boot the **python** micro framework for building web applications.: atom:,html presentation framework the swift programming language spring boot the python micro framework for building web applications . : atom :,python,None,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...
12292,-0.009809,-0.004734,er base images frappe _ mirror ddd like visualization plugin for qt creator. ( 2011,##er base images fra ##ppe _ mirror dd ##d like visual ##ization plug ##in for q ##t creator . ( 2011,visual,kazupon/vue-i18n,:globe_with_meridians: Internationalization plugin for Vue.js,6234.0,767.0
12293,-0.009166,-0.010862,"4j, datavec and more-deep **learning** & linear algebra for java / scala with gpus","##4 ##j , data ##ve ##c and more - deep learning & linear algebra for java / scala with gp ##us",learning,eclipse/deeplearning4j,"Eclipse Deeplearning4j, ND4J, DataVec and more - deep learning & linear algebra for Java/Scala with GPUs + Spark",12109.0,4895.0
12294,-0.007962,-0.008329,[UNK] [UNK] [UNK] spring [UNK] [UNK] [UNK] [UNK] spring source **learning** 5. 2. 9. release spring [UNK] [UNK],[UNK] [UNK] [UNK] spring [UNK] [UNK] [UNK] [UNK] spring source learning 5 . 2 . 9 . release spring [UNK] [UNK],learning,LearningLocker/learninglocker,Learning Locker - The Open Source Learning Record Store,448.0,213.0
12295,-0.009228,-0.009300,example code and exercises from textbook 'hands-on **machine** learning with scikit-learn & tensorflow ',example code and exercises from textbook ' hands - on machine learning with sci ##kit - learn & tensor ##flow ',machine,tensorflow/tensorflow,An Open Source Machine Learning Framework for Everyone,156754.0,84937.0


In [66]:
#df.to_csv(r'/home/zz3hs/git/dspg21oss/data/dspg21oss/word_embedding_viz.csv', index = False)

In [4]:
df =pd.read_csv(r'/home/zz3hs/git/dspg21oss/data/dspg21oss/word_embedding_viz.csv')

In [12]:
df_sub[df_sub["slug"] == "redis/redis"]

,x,y,context,tokens,word,slug,description,topic,stars
6,-0.008622,-0.005464,completely ignore. redis is an in-memory **database** that persists on disk. the data model is,completely ignore . red ##is is an in - memory database that persist ##s on disk . the data model is,database,redis/redis,"Redis is an in-memory database that persists on disk. The data model is key-value, but many different kind of values are supported: Strings, Lists, Sets, Sorted Sets, Hashes, Streams, HyperLogLogs...",49559.0,19455.0
4388,-0.009128,-0.005171,spheric and planetary physics wannabe main memory **database** and more documentation for crafttweaker documentation for,##sp ##her ##ic and planetary physics wanna ##be main memory database and more documentation for craft ##t ##we ##aker documentation for,database,redis/redis,"Redis is an in-memory database that persists on disk. The data model is key-value, but many different kind of values are supported: Strings, Lists, Sets, Sorted Sets, Hashes, Streams, HyperLogLogs...",49559.0,19455.0
5914,-0.009118,-0.006291,ub interface and adds useful features clustered in-memory **database** based on mnesia with simple sql layer the,##ub interface and adds useful features clustered in - memory database based on mn ##es ##ia with simple sql layer the,database,redis/redis,"Redis is an in-memory database that persists on disk. The data model is key-value, but many different kind of values are supported: Strings, Lists, Sets, Sorted Sets, Hashes, Streams, HyperLogLogs...",49559.0,19455.0


In [18]:
x = ["machine","learning", "ml"]

df_sub = df[df.word.isin(x)]

# Make the plot
alt.Chart(df_sub, title=f"Word Similarity: AI, ML, database, visualization").mark_circle(size=200).encode(
    alt.X('x',
        scale=alt.Scale(zero=False)
    ), y="y",
    color= 'word',
    #href="link",
    tooltip=['slug', 'description', 'topic', 'stars']
    ).interactive().properties(
    width=500,
    height=500
)

alt.Chart(...)

In [81]:
# Make the plot
alt.Chart(df_sub, title=f"Word Similarity: AI, ML, database, visualization").mark_circle(size=100).encode(
    alt.X('x',
        scale=alt.Scale(zero=False)
    ), y="y",
    color= color_by,
    size=alt.Size('stars'),
    #href="link",
    tooltip=['slug', 'description', 'topic', 'stars']
    ).interactive().properties(
    width=500,
    height=500
)


alt.Chart(...)

In [45]:
#5000 Record

# Rename columns so that the context shows up as the "title" in the tooltip (bigger and bolded)
#df = df.rename(columns={'slug': 'repo_slug', 'context': 'description'})

# Make the plot
alt.Chart(df, title=f"Word Similarity: Python, R").mark_circle(size=200).encode(
    alt.X('x',
        scale=alt.Scale(zero=False)
    ), y="y",
    color= color_by,
    #href="link",
    tooltip=['slug', 'description', 'topic', 'stars']
    ).interactive().properties(
    width=500,
    height=500
)

alt.Chart(...)